<a href="https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/script/code-snippet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Auto-move file 
#@markdown This code automatically moves the `last.ckpt` file, the `last-state` folder, the `train_data`
#@markdown directory, and the `meta_lat.json` directory from the output directory to the cloned model
#@markdown and datasets repositories. Before running this code, you need to clone the datasets and
#@markdown model repositories from huggingface. The code checks for the existence of these files and
#@markdown folders in the source and destination directories, and prints messages if they already
#@markdown exist or do not exist. It uses the os and shutil libraries to check for the existence of
#@markdown files and folders and to move them.
import shutil

# The path of the output directory
output_dir = '/content/kohya-trainer/fine_tuned/' #@param {'type':'string'}

# The name of the model
model_name = 'momoko30k' #@param {'type':'string'}

# The path of the cloned model repository
cloned_model_repo = '/content/momoko' #@param {'type':'string'}

# The name of the save state
save_state_name = 'momoko30k-state' #@param {'type':'string'}

# The path of the cloned datasets repository
cloned_datasets_repo = '/content/momoko-tag' #@param {'type':'string'}

# The path of the meta lat json directory
meta_lat_json_dir = "/content/kohya-trainer/meta_lat.json" #@param {'type':'string'}

# The path of the train data directory
train_data_dir = "/content/kohya-trainer/train_data" #@param {'type':'string'}

if opt_out == True :
    # Move file
  src_file = f'{output_dir}/last.ckpt'
  dst_file = f'{cloned_model_repo}/{model_name}.ckpt'
  if os.path.exists(src_file):
      if not os.path.exists(dst_file):
          shutil.move(src_file, dst_file)
          print(f'Moved {src_file} to {dst_file}\n', flush=True)
      else:
          print(f'{dst_file} already exists\n', flush=True)
  else:
      print(f'There is no {src_file} like that\n', flush=True)

  # Move folder
  src_folder = f'{output_dir}/last-state'
  dst_folder = f'{cloned_datasets_repo}/{save_state_name}'
  if os.path.exists(src_folder):
      if not os.path.exists(dst_folder):
          shutil.move(src_folder, dst_folder)
          print(f'Moved {src_folder} to {dst_folder}\n', flush=True)
      else:
          print(f'{dst_folder} already exists\n', flush=True)
  else:
      print(f'There is no {src_folder} like that\n', flush=True)

  # Define train data directory
  dst_train_data_dir = f'{cloned_datasets_repo}/train_data'

  # Check if train data directory already exists
  if not os.path.exists(dst_train_data_dir):
    # Move train data directory
    src_train_data_dir = f'{train_data_dir}'
    if os.path.exists(src_train_data_dir):
        shutil.move(src_train_data_dir, dst_train_data_dir)
        print(f'Moved {src_train_data_dir} to {dst_train_data_dir}\n', flush=True)
    else:
        print(f'There is no {src_train_data_dir} like that\n', flush=True)

  # Define meta lat json directory
  dst_meta_lat_json_dir = f'{cloned_datasets_repo}/meta_lat.json'

  # Check if meta lat json directory already exists
  if not os.path.exists(dst_meta_lat_json_dir):
    # Move meta lat json directory
    src_meta_lat_json_dir = f'{meta_lat_json_dir}'
    if os.path.exists(src_meta_lat_json_dir):
        shutil.move(src_meta_lat_json_dir, dst_meta_lat_json_dir)
        print(f'Moved {src_meta_lat_json_dir} to {dst_meta_lat_json_dir}\n', flush=True)
    else:
        print(f'There is no {src_meta_lat_json_dir} like that\n', flush=True)

  # Iterate over all files and folders in the cloned_datasets_repo directory
  for filename in os.listdir(cloned_datasets_repo):
    # Check if the file or folder is not the save_state_name folder, the train_data folder, or the meta_lat.json file
    if filename != save_state_name and filename != os.path.basename(dst_train_data_dir) and filename != os.path.basename(dst_meta_lat_json_dir):
      # Get the path of the file or folder
      file_path = os.path.join(cloned_datasets_repo, filename)

      # Check if the file or folder is a directory (i.e., a folder)
      if os.path.isdir(file_path):
        # Delete the folder
        shutil.rmtree(file_path)
        print(f'Deleted folder: {filename}')
      else:
        # Delete the file
        os.remove(file_path)
        print(f'Deleted file: {filename}')

